In [1]:
import pandas as pd

In [2]:
caminho_csv = "/mnt/c/Users/AcerGamer/Downloads/Tuberculose/Projeto-Tuberculose/dados/arquivos_csv/dados_tuberculose_completos.csv"

In [3]:
df = pd.read_csv(caminho_csv, encoding='latin1', low_memory=False)

In [4]:
# Remoção de colunas com 100% de valores ausentes ou irrelevantes para modelagem

In [5]:
colunas_para_remover = [
    'DT_TRANSRM', 'CS_FLXRET', 'FLXRECEBI', 'MIGRADO_W', 'ID_OCUPA_N',
    'RIFAMPICIN', 'ISONIAZIDA', 'ETAMBUTOL', 'ESTREPTOMI', 'PIRAZINAMI',
    'ETIONAMIDA', 'OUTRAS', 'OUTRAS_DES', 'DT_MUDANCA', 'SITUA_12_M'
]

In [6]:
df.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

In [7]:
# Conversão de colunas de data

In [8]:
colunas_data = ['DT_NOTIFIC', 'DT_DIAG', 'DT_DIGITA', 'DT_INIC_TR', 'DT_NOTI_AT', 'DT_ENCERRA']
for col in colunas_data:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [9]:
# Decodificação da idade em anos

In [10]:
def decodifica_idade(row):
    if pd.isnull(row['NU_IDADE_N']):
        return None
    idade = int(row['NU_IDADE_N'])
    if idade < 4000:
        return idade  # Assume anos diretos
    tipo = idade // 1000
    valor = idade % 1000
    if tipo == 1:
        return round(valor / 365, 1)  # dias
    elif tipo == 2:
        return round(valor / 12, 1)   # meses
    elif tipo == 3:
        return valor                 # anos
    return None

In [11]:
if 'NU_IDADE_N' in df.columns:
    df['NU_IDADE_ANOS'] = df.apply(decodifica_idade, axis=1)

In [12]:
# Decodificar variáveis categóricas com mapeamentos conhecidos

In [13]:
sexo_map = {'M': 'Masculino', 'F': 'Feminino'}
gestante_map = {
    1: '1º Trimestre', 2: '2º Trimestre', 3: '3º Trimestre',
    4: 'Idade gestacional ignorada', 5: 'Não', 6: 'Não se aplica', 9: 'Ignorado'
}
raca_map = {
    1: 'Branca', 2: 'Preta', 3: 'Amarela', 4: 'Parda', 5: 'Indígena', 9: 'Ignorado'
}
escolaridade_map = {
    0: 'Sem escolaridade', 1: 'Fundamental Incompleto', 2: 'Fundamental Completo',
    3: 'Médio Incompleto', 4: 'Médio Completo', 5: 'Superior Incompleto',
    6: 'Superior Completo', 9: 'Ignorado'
}
tratamento_map = {
    1: 'Esquema Básico', 2: 'Esquema Especial', 3: 'Esquema para Recaída',
    4: 'Esquema para Retorno', 5: 'Outro', 9: 'Ignorado'
}
hiv_map = {1: 'Reagente', 2: 'Não Reagente', 3: 'Em andamento', 4: 'Não realizado', 9: 'Ignorado'}
doenca_tra_map = {1: 'Sim', 2: 'Não', 9: 'Ignorado'}

In [14]:
# Converte para numérico (caso seja string) antes de mapear
#df['CS_SEXO'] = pd.to_numeric(df['CS_SEXO'], errors='coerce').map(sexo_map)
df['CS_GESTANT'] = pd.to_numeric(df['CS_GESTANT'], errors='coerce').map(gestante_map)
df['CS_RACA'] = pd.to_numeric(df['CS_RACA'], errors='coerce').map(raca_map)
df['CS_ESCOL_N'] = pd.to_numeric(df['CS_ESCOL_N'], errors='coerce').map(escolaridade_map)
df['TRATAMENTO'] = pd.to_numeric(df['TRATAMENTO'], errors='coerce').map(tratamento_map)
df['DOENCA_TRA'] = pd.to_numeric(df['DOENCA_TRA'], errors='coerce').map(doenca_tra_map)
df['HIV'] = pd.to_numeric(df['HIV'], errors='coerce').map(hiv_map)

In [15]:
# Salvar o arquivo tratado

In [16]:
saida = "dados_tuberculose_transformados.csv"
df.to_csv(saida, index=False)
print(f"Arquivo salvo como {saida} com {df.shape[0]} linhas e {df.shape[1]} colunas.")

Arquivo salvo como dados_tuberculose_transformados.csv com 502271 linhas e 83 colunas.
